In [1]:
!pip install boto3



Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'  

In [3]:
import boto3
kinesis = boto3.client('kinesis', region_name='us-east-1')

try:
    response = kinesis.describe_stream(StreamName="transaction-stream")
    print("Stream exists! Status:", response['StreamDescription']['StreamStatus'])
except kinesis.exceptions.ResourceNotFoundException:
    print("Stream does NOT exist - creation failed")

Stream exists! Status: ACTIVE


In [ ]:
import boto3
import json
import random
import time
from faker import Faker
from datetime import datetime

fake = Faker()
kinesis = boto3.client('kinesis', region_name='us-east-1')  

def generate_transaction():
    return {
        "user_id": fake.uuid4(),
        "timestamp": datetime.utcnow().isoformat(),
        "amount": round(random.uniform(5.00, 10000.00), 2),
        "city": fake.city(),  
        "country": fake.country(),  
        "state": fake.state() if hasattr(fake, 'state') else fake.state_abbr(),  
        "zipcode": fake.zipcode(),
        "device": random.choice(['mobile', 'web', 'POS']),
        "merchant": fake.company(),
        "card_type": random.choice(['Visa', 'MasterCard', 'Amex']),
    }

while True:
    txn = generate_transaction()
    print(f"Sending transaction: {txn}")
    try:
        response = kinesis.put_record(
            StreamName="transaction-stream",
            Data=json.dumps(txn),
            PartitionKey=txn['user_id']
        )
        print(f"Success! Sequence: {response['SequenceNumber']}")
    except Exception as e:
        print(f"Error sending: {e}")
    time.sleep(1)

Sending transaction: {'user_id': 'bc29d96e-83a1-48ce-bf63-398e73aab30f', 'timestamp': '2025-07-23T14:13:39.321316', 'amount': 2978.38, 'city': 'Lanceside', 'country': 'Sao Tome and Principe', 'state': 'Vermont', 'zipcode': '10544', 'device': 'mobile', 'merchant': 'Love and Sons', 'card_type': 'MasterCard'}
Success! Sequence: 49665424195717824169799966322607894632099924764551806978
Sending transaction: {'user_id': '8a4b5e1a-efca-4822-acf2-7dacae172d7e', 'timestamp': '2025-07-23T14:13:44.102257', 'amount': 4341.15, 'city': 'Tonyastad', 'country': 'Heard Island and McDonald Islands', 'state': 'South Carolina', 'zipcode': '98042', 'device': 'mobile', 'merchant': 'Logan-Allen', 'card_type': 'Visa'}
Success! Sequence: 49665424195717824169799966324225437378744298737747623938
Sending transaction: {'user_id': '0d8c27ed-e63b-41e9-ad4e-11294c4d2b22', 'timestamp': '2025-07-23T14:13:45.686858', 'amount': 9580.67, 'city': 'New Douglas', 'country': 'Chad', 'state': 'Rhode Island', 'zipcode': '21211',